# 1. Collect and explore the data

## Goals

This notebook aims to explore, clean, and structure the options data that will be used in the pricing modeling project (Black-Scholes, Heston, etc.).

Therefore, this notebook analyzes:
- Structure and characteristic of the option market-
- Relevant filters to ensure data quality
- Calculation of useful features as `mid price`, `spread`, `time to maturity (T)` and `interest free rate`

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
from mpl_toolkits.mplot3d import Axes3D

sns.set(style="darkgrid")
%matplotlib inline

## 1.1. Downloading the data.

In this notebook section, I will download the data from YahooFinance. The financial asset choose is the `^SPX` ticker. The time to maturity is also computed.

In [2]:
ticker_symbol = "^SPX"
ticker = yf.Ticker(ticker_symbol)

spot_price = ticker.history(period="1d")['Close'].iloc[-1]

expirations = ticker.options
print(f"Available expirations from {expirations[0]} to {expirations[len(expirations)-1]}")

Available expirations from 2025-07-30 to 2030-12-20


In [3]:
options_df = []

for expiry in expirations[1:]:
    try:
        option_chain = ticker.option_chain(expiry)
        expiry_date = datetime.strptime(expiry, "%Y-%m-%d")
        T = (expiry_date - datetime.today()).days/365.0
        

        for df, opt_type in zip([option_chain.calls, option_chain.puts], ["call", "put"]):
            df = df.copy()
            df["Type"] = opt_type
            df["T"] = T
            options_df.append(df)
        
    except Exception as e:
        print(f"Error with expriry {expiry}: e")

options_df = pd.concat(options_df, ignore_index=True)
options_df.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,Type,T
0,SPXW250731C01400000,2025-07-14 19:24:49+00:00,1400.0,4868.23,4971.1,4990.3,0.0,0.0,2.0,1432.0,8.888188,True,REGULAR,USD,call,0.0
1,SPXW250731C02600000,2025-07-28 14:14:27+00:00,2600.0,3798.22,3774.5,3792.8,0.0,0.0,1.0,1.0,5.665408,True,REGULAR,USD,call,0.0
2,SPXW250731C03900000,2025-05-07 18:00:59+00:00,3900.0,1749.80,2108.9,2136.9,0.0,0.0,4.0,4.0,0.000010,True,REGULAR,USD,call,0.0
3,SPXW250731C04000000,2025-07-10 19:01:07+00:00,4000.0,2292.49,2376.7,2383.4,0.0,0.0,1.0,108.0,2.929446,True,REGULAR,USD,call,0.0
4,SPXW250731C04050000,2025-06-24 14:21:37+00:00,4050.0,2037.32,2330.3,2354.1,0.0,0.0,6.0,7.0,3.408449,True,REGULAR,USD,call,0.0


## 1.2. Data preprocessing

In this section, I apply relevant filters to guarantee that: 
- Options have enough liquidity (`openInterest > X`)
- Do not have inconsistencies (`bid > ask`)
- Options are **vanilla**

The `mid price` y `spread` are also computed for each action. The `interest free rate` is interpolated from USA treasure bill interest.

In [4]:
def filter_options(df, min_open_interest=100):
    useful_cols = ['strike', 'bid', 'ask', 'lastPrice', 'openInterest', 'impliedVolatility', 'Type', 'T']
    df = df[useful_cols].copy()
    df = df[df['openInterest'] > min_open_interest]
    df = df[df['bid'] > 0]
    df = df[df['ask'] > 0]
    df = df[df['bid'] < df['ask']]
    df["mid"] = (df["bid"] + df["ask"]) / 2
    df["spread"] = df["ask"] - df["bid"]
    return df

options_df = filter_options(options_df)
print(options_df.shape)

(7030, 10)


In [5]:
#interest free rates 29-july-2025
tenors = np.array([0.25, 0.5, 1.0, 2.0, 5.0])
rates = np.array([0.0433, 0.0426, 0.0408, 0.0391, 0.0396])

def interp_rate(T, tenors, rates):
    return np.interp(T, tenors, rates)

options_df['risk_free_rate'] = options_df['T'].apply(lambda T: interp_rate(T, tenors, rates))
options_df['Underlying_price'] = spot_price
print(options_df.shape[0])
options_df.head()

NameError: name 'options_ds' is not defined

## 1.3. Initial visulatization

Through a scatter plot, this section allow me to visualize the implied volatility estimated by `YahooFinance`.

In [ ]:
call_options = options_df[options_df['Type'] == 'call'].copy()

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

sc = ax.scatter(
    call_options['T'],                    
    call_options['strike'],                
    call_options['impliedVolatility'],   
    c=call_options['openInterest'],      
    cmap='viridis',
    marker='o',
    alpha=0.8,
    vmin=0,     
    vmax=1000 
)

ax.set_xlabel('Time to Maturity (T)')
ax.set_ylabel('Strike Price')
ax.set_zlabel('Implied Volatility')

cbar = plt.colorbar(sc, pad=0.1)
cbar.set_label('Open Interest')

plt.tight_layout()
plt.show()


## 1.4. Data saving

This section saves the data to a `.csv` file in the `Data/` directory.

In [ ]:
output_path = "../data/options_cleaned.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
options_df.to_csv(output_path, index=False)
print(f"Saved cleaned option chain to: {output_path}")

# 1.5. Summary

- Data collected and stored for multiple S&P500 options.
- Initial preprocessing discarding non-relevant data.
- Calc useful metrics such as related interest free rate and time to maturity.
- Saved to ../data/options_cleaned.csv for further preprocessing.

The next step is to apply different models (e.g. Black-Scholes or Heston model) to option pricing and hedge strategies. These will be done in following notebooks.